# Huggingface Summarization of Patent documents using Fine Tuned T5-Base with Big_Patent

## Installation

_**Note:** The use of Jupyter is optional: We could also launch SageMaker Training jobs from anywhere we have an SDK installed, connectivity to the cloud and appropriate permissions, such as a Laptop, another IDE or a task scheduler like Airflow or AWS Step Functions._

In [30]:
!pip install "sagemaker>=2.48.0" "transformers==4.6.1" "datasets[s3]==1.6.2" --upgrade
#!apt install git-lfs

  Using cached sagemaker-2.73.0.tar.gz (481 kB)
  Preparing metadata (setup.py) ... done
  Using cached sagemaker-2.72.3.tar.gz (475 kB)
  Preparing metadata (setup.py) ... done
  Using cached sagemaker-2.72.2.tar.gz (473 kB)
  Preparing metadata (setup.py) ... done
  Using cached sagemaker-2.72.1-py2.py3-none-any.whl
  Using cached sagemaker-2.72.0.tar.gz (477 kB)
  Preparing metadata (setup.py) ... done
  Using cached sagemaker-2.71.0.tar.gz (477 kB)
  Preparing metadata (setup.py) ... done
  Using cached sagemaker-2.70.0.tar.gz (466 kB)
  Preparing metadata (setup.py) ... done


In [31]:
!curl -s https://packagecloud.io/install/repositories/github/git-lfs/script.rpm.sh | sudo bash
!sudo yum install git-lfs -y
!git lfs install

Detected operating system as amzn/2018.
Checking for curl...
Detected curl...
done.
Installing pygpgme to verify GPG signatures...
Loaded plugins: dkms-build-requires, priorities, update-motd, upgrade-helper,
              : versionlock
amzn-main                                                | 2.1 kB     00:00     
amzn-updates                                             | 3.8 kB     00:00     
copr:copr.fedorainfracloud.org:vbatts:shadow-utils-newxi | 3.3 kB     00:00     
github_git-lfs-source/signature                          |  833 B     00:00     
github_git-lfs-source/signature                          | 1.8 kB     00:00 !!! 
libnvidia-container/x86_64/signature                     |  833 B     00:00     
libnvidia-container/x86_64/signature                     | 2.1 kB     00:00 !!! 
nvidia-container-runtime/x86_64/signature                |  833 B     00:00     
nvidia-container-runtime/x86_64/signature                | 2.1 kB     00:00 !!! 
nvidia-docker/x86_64/signature    

In [32]:
import sagemaker.huggingface

## Config

_If you are going to use Sagemaker in a local environment. You need access to an IAM Role with the required permissions for Sagemaker. You can find [here](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-roles.html) more about it._

In [33]:
import sagemaker

sess = sagemaker.Session()
role = sagemaker.get_execution_role()

print(f"IAM role arn used for running training: {role}")
print(f"S3 bucket used for storing artifacts: {sess.default_bucket()}")

IAM role arn used for running training: arn:aws:iam::859830842924:role/service-role/AmazonSageMaker-ExecutionRole-20211013T235227
S3 bucket used for storing artifacts: sagemaker-us-east-1-859830842924


In [34]:
git_config = {'repo': 'https://github.com/huggingface/transformers.git','branch': 'v4.6.1'} # v4.6.1 is referring to the `transformers_version` you use in the estimator.

## Configure distributed training and hyperparameters

Next, we will define our `hyperparameters` and configure our distributed training strategy. As hyperparameter, we can define any [Seq2SeqTrainingArguments](https://huggingface.co/transformers/main_classes/trainer.html#seq2seqtrainingarguments) and the ones defined in [run_summarization.py](https://github.com/huggingface/transformers/tree/master/examples/seq2seq#sequence-to-sequence-training-and-evaluation). 

In [47]:
# Fine Tune T5-base model with big_patent dataset

# hyperparameters, which are passed into the training job
hyperparameters={'per_device_train_batch_size': 2,
                 'per_device_eval_batch_size': 2,
                 'model_name_or_path': 't5-base', # https://huggingface.co/t5-base
                 'dataset_name': 'xsum', 
                 # goal is to train using Big_Patent, currently doing xsum as it's smaller/faster.
                 # https://huggingface.co/datasets/big_patent
                 # 'dataset_name': 'big_patent',
                 # 'dataset_config_name': 'g',
                 'do_train': True,
                 'do_eval': True,
                 #'do_predict': True, # not using while testing
                 #'predict_with_generate': True, # not using while testing
                 # 'output_dir': '/opt/ml/checkpoints', # defaults to '/opt/ml/model', results in output tar.gz containing all checkpoint files.
                 'num_train_epochs': 1,
                 'learning_rate': 5e-5,
                 'seed': 7,
                 'fp16': True,
                 #'cache_dir': '/tmp',
                 # for big_patent...
                 # 'text_column': 'description',
                 # 'summary_column': 'abstract',
                 # for xsum...
                 'text_column': 'document',
                 'summary_column': 'summary',
                 #'enable_checkpointing' = False, # errors, must not be a HP of the HF trainer??? 
                 'save_total_limit': 1 # adding to test remove all prev checkpoints from output file - https://huggingface.co/docs/transformers/v4.15.0/en/main_classes/trainer#transformers.TrainingArguments
                }

# configuration for running training on smdistributed Data Parallel
distribution = {'smdistributed':{'dataparallel':{ 'enabled': True }}}

# set S3 output location for checkpoints
checkpoint_s3_uri=f's3://{sess.default_bucket()}/checkpoints'

## Create a `HuggingFace` estimator and start training

In [48]:
from sagemaker.huggingface import HuggingFace

# create the Estimator
huggingface_estimator = HuggingFace(
      entry_point='run_summarization.py', # script
      source_dir='./examples/pytorch/summarization', # relative path to example
      git_config=git_config,
      instance_type='ml.p3.16xlarge',
      instance_count=2,
      transformers_version='4.6.1',
      pytorch_version='1.7.1',
      py_version='py36',
      role=role,
      hyperparameters = hyperparameters,
      distribution = distribution,
      volume_size = 400, # allocating ridiculous amount of EBS so checkpoints don't kill job.
      enable_checkpointing = False, # does not work for HF Estimator it seems
      checkpoint_s3_uri = checkpoint_s3_uri # doesn't seem to help, checkpoints still stored
      # cache_dir='/tmp' # doesn't seem to help, checkpoints still stored
)

In [ ]:
# starting the train job
huggingface_estimator.fit()
# save model to
huggingface_estimator.save_model('/opt/ml/model')

## Deploying the endpoint from Estimator

To deploy our endpoint, we call `deploy()` on our HuggingFace estimator object, passing in our desired number of instances and instance type.

In [ ]:
predictor = huggingface_estimator.deploy(1,"ml.g4dn.xlarge")

## Deploying the endpoint from Saved Model

In [ ]:
# create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
   model_data="s3://sagemaker-us-east-1-859830842924/huggingface-pytorch-training-2022-01-24-12-30-55-527/output/model.tar.gz",  # path to your trained SageMaker model
   role=role,                                            # IAM role with permissions to create an endpoint
   transformers_version="4.6",                           # Transformers version used
   pytorch_version="1.7",                                # PyTorch version used
   py_version='py36',                                    # Python version used
)

# deploy model to SageMaker Inference
predictor = huggingface_model.deploy(
   initial_instance_count=1,
   instance_type="ml.m5.xlarge"
)

Then, we use the returned predictor object to call the endpoint.

In [ ]:
patent = '''BACKGROUND OF THE DISCLOSURE

      1. Technical Field of the Disclosure
      This embodiment relates in general to electric cars. More specifically, the preferred embodiment relates to an electric car having a vertical row of seats.
      2. Description of the Related Art
      An electric car is powered by an electric motor using electrical energy stored in batteries or other charged devices. Electric cars are environment friendly as it do not produce any harmful gases such as carbon monoxide, organic compounds, hydro carbons etc. Electric cars are economical because of very low maintenance and operating costs.
      Conventional electric cars have considerable drawbacks. For example, an existing electric car includes an electric drive and at least one connected electrical energy storage module. A guide extends longitudinally along the motor vehicle, and supports the storage module in a manner relative to the motor vehicle. Conventional electric cars employ horizontal rows of seats consuming excess space.
      Another existing electric vehicle is capable of carrying at least two passengers and has at least three wheels. Passengers sit in tandem and most of the batteries or fuel cell systems are located to the sides of the passengers. This vehicle has an aerodynamically shaped body with substantially reduced frontal area and drag. The body is lightweight, made from shock absorbing materials and structures, and has pressure-airless tires, which enhances the safety of the passengers. The vehicle also includes an advanced hydrogen-electric hybrid propulsion system with quick refueling from existing infrastructure and various additional optional features and systems. However the batteries or fuel cell systems are placed to the sides of the passengers which increases the overall the dimension and weight of the vehicle.
      Yet another existing electrical car embodiment is comprised of bodywork with ground-engaging wheels for vehicle motion over the ground, the bodywork contains an electric motor to drive the vehicle via the wheels and batteries to power the electric motor.
      This embodiment provides an additional energy generation means comprised of a tunnel extending through the bodywork. It includes a turbine fan/alternator set located in the tunnel at the rear of the vehicle where electrical energy is generated during vehicle motion to charge the batteries. This results in improved performance of the vehicle, especially with regard to its range. The inlet to the tunnel at the vehicle front constitutes the major portion of the vehicles frontal area. A special alternator is provided, and the vehicle can also include a solar cell means for battery charging. However, the seats are placed far apart which consumes a lot of space.
      Hence, it can be seen, that there is a need for an electric car that contains a vertical rows of seats. This needed electric car would save more space than existing models to form a vehicle of smaller size with more passengers. Moreover, this needed electric car would consume less power and use a less bulky charging means during transportation. The present embodiment accomplishes these objectives.
SUMMARY OF THE DISCLOSURE

      To minimize the limitations found in the prior art, and to minimize other limitations that will be apparent upon the reading of the specifications, the present invention provides an electric car having a vertical row of seat for accommodating at least one passenger. The electric car comprises a center console placed at an interior forepart of the electric car. A steering is attached to the center console. A rectangular seat is mounted on a rectangular box, the rectangular box being longitudinally placed at a center part of the electric car. A storage area is enclosed within the rectangular box, the storage area includes a personal storage and a battery storage. The battery storage includes a battery pack having a set of rechargeable batteries. The battery pack is used to power the electric car. A plug point is located at a rear end of the electric car for charging the battery pack. A pair of rotatable front wheels and a pair of rotatable back wheels are provided for ensuring smooth movement of the electric car. Both the center console and the steering of the electric car are aligned with the rectangular seat. The rectangular seat is mounted on the rectangular box having the storage area in order to save space. The electric car is configured to have a compact seating arrangement. A back support may be provided for a driver's rectangular seat. The electric car is specially designed to achieve better performance by increasing the energy efficiency and reducing the power consumption. The at least one passenger can be seated facing front direction with one leg on each side of the rectangular seat. The electric car is capable of accommodating more passengers with relatively small size and with minimum power consumption during transportation.
      In alternate embodiment of the present invention at least one step and ladder seat is positioned at the center part of the electric car. The electric car comprises a car body having a front end, a rear end, a top portion and a bottom portion. A center console placed at an interior forepart of the electric car. A steering is attached to the center console. At least one step and ladder seat is positioned at the center part of the electric car for accommodating at least one passenger. In addition to the at least one step and ladder seat, a horizontal row of seats for accommodating a pair of passengers is located at the rear end of the vehicle. A battery storage having a battery pack is positioned at the bottom portion of step and ladder seat for powering the electric car. A plug point is located at a rear end of the electric car for charging the battery pack. A personal storage is placed at the rear end of horizontal row of seat.
      One objective of the invention is to provide an electric car capable of accommodating four or more passengers with relatively smaller vehicle size.
      Another objective of the invention is to provide an electric car with compact seating arrangement.
      A third objective of the invention is to provide an electric car with better performance by increasing the energy efficiency and reducing the power consumption.
      These and other advantages and features of the present invention are described with specificity so as to make the present invention understandable to one of ordinary skill in the art.'''

data= {"inputs":patent}

predictor.predict(data)

Finally, we delete the endpoint again.

In [ ]:
predictor.delete_endpoint()